### Imports

##### Libraries

In [1]:
import requests
import pandas as pd
from pandas import json_normalize

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

##### API Access Credentials

In [3]:
from access_details import CLIENT_ID_
from access_details import CLIENT_SECRET_

## Dubai Communities Wikipedia Dataset

##### Import Data from Wikipedia

In [4]:
path = "https://en.wikipedia.org/wiki/List_of_communities_in_Dubai"

In [5]:
wiki_df = pd.read_html(path, header=0)[0]

In [6]:
wiki_df.head()

,Community Number,Community (English),Community (Arabic),Area(km2),Population(2000),Population density(/km2),Unnamed: 6
0,126.0,Abu Hail,أبو هيل,1.27 km²,21414,"16,861.4/km²",NaN
1,711.0,Al Awir First,العوير الأولى,NaN,NaN,NaN,NaN
2,721.0,Al Awir Second,العوير الثانية,NaN,NaN,NaN,NaN
3,283.0,Aleyas,العياص,162.4 km2,1706,162.4/km2,NaN
4,333.0,Al Bada'a,البدع,0.82 km²,18816,22946/km²,NaN


In [7]:
wiki_df.describe()

,Community Number,Unnamed: 6
count,145.000000,0.0
mean,384.068966,NaN
std,218.663258,NaN
min,50.000000,NaN
25%,244.000000,NaN
50%,337.000000,NaN
75%,422.000000,NaN
max,971.000000,NaN


### Data Preprocessing

##### Drop Columns

In [8]:
wiki_df.columns

Index(['Community Number', 'Community (English)', 'Community (Arabic)',
       'Area(km2)', 'Population(2000)', 'Population density(/km2)',
       'Unnamed: 6'],
      dtype='object')

In [9]:
#Drop population columns as data is 20 years old
drop_columns = ['Community (Arabic)', 'Population(2000)', 'Population density(/km2)',
       'Unnamed: 6']
wiki_df = wiki_df.drop(drop_columns, axis=1)

In [10]:
wiki_df.head()

,Community Number,Community (English),Area(km2)
0,126.0,Abu Hail,1.27 km²
1,711.0,Al Awir First,NaN
2,721.0,Al Awir Second,NaN
3,283.0,Aleyas,162.4 km2
4,333.0,Al Bada'a,0.82 km²


#### Rename Columns

In [11]:
wiki_df = wiki_df.rename(columns = {'Community (English)': 'Community Name'})

In [12]:
wiki_df.head()

,Community Number,Community Name,Area(km2)
0,126.0,Abu Hail,1.27 km²
1,711.0,Al Awir First,NaN
2,721.0,Al Awir Second,NaN
3,283.0,Aleyas,162.4 km2
4,333.0,Al Bada'a,0.82 km²


#### Remove text from Area column

In [13]:
wiki_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 146 entries, 0 to 145
Data columns (total 3 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Community Number  145 non-null    float64
 1   Community Name    146 non-null    object 
 2   Area(km2)         112 non-null    object 
dtypes: float64(1), object(2)
memory usage: 3.5+ KB


In [14]:
wiki_df['Area(km2)'] = wiki_df['Area(km2)'].str.replace('k', ' ').str.split(' ', expand=True)

In [15]:
wiki_df.head()

,Community Number,Community Name,Area(km2)
0,126.0,Abu Hail,1.27
1,711.0,Al Awir First,NaN
2,721.0,Al Awir Second,NaN
3,283.0,Aleyas,162.4
4,333.0,Al Bada'a,0.82


#### Recast column types

In [16]:
nan_community_number = wiki_df.loc[wiki_df['Community Name'] == 'Downtown Dubai']
nan_community_number

,Community Number,Community Name,Area(km2)
84,NaN,Downtown Dubai,2.68


In [17]:
wiki_df = wiki_df.drop(nan_community_number.index, axis = 0)

In [18]:
nan_community_number = wiki_df.loc[wiki_df['Community Name'] == 'Downtown Dubai']
nan_community_number

,Community Number,Community Name,Area(km2)


In [19]:
wiki_df = wiki_df.astype({"Community Number":'int64', "Area(km2)":'float'}) 

In [20]:
wiki_df.head()

,Community Number,Community Name,Area(km2)
0,126,Abu Hail,1.27
1,711,Al Awir First,NaN
2,721,Al Awir Second,NaN
3,283,Aleyas,162.40
4,333,Al Bada'a,0.82


### Save Dataset as CSV

In [21]:
path = 'wiki_dubai_communities.csv'
wiki_df.to_csv(path, index=False)

## Dubai FoursquareAPI Venue Dataset

#### Import Data JSON from foursquare API

##### API Access Details

In [22]:
CLIENT_ID = CLIENT_ID_
CLIENT_SECRET = CLIENT_SECRET_
VERSION = '20201120'

##### Parameters

In [23]:
lat, long = 25.276987, 55.296249
radius = 100000
LIMIT = 100000

##### Define URL & Get JSON

In [24]:
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}' \
.format(CLIENT_ID, CLIENT_SECRET, lat, long, VERSION, radius, LIMIT)

In [25]:
results = requests.get(url).json()

### JSON to Pandas

In [26]:
venues_df = results['response']['groups'][0]['items']
venues_df = json_normalize(venues_df)
venues_df.head(3)

,referralId,reasons.count,reasons.items,venue.id,venue.name,venue.location.address,venue.location.lat,venue.location.lng,venue.location.labeledLatLngs,venue.location.distance,venue.location.cc,venue.location.neighborhood,venue.location.city,venue.location.state,venue.location.country,venue.location.formattedAddress,venue.categories,venue.photos.count,venue.photos.groups,venue.location.crossStreet,venue.location.postalCode,venue.venuePage.id
0,e-0-508e4b2ae4b01ceeb4ae3d37-0,0,"[{'summary': 'This spot is popular', 'type': 'general', 'reasonName': 'globalInteractionReason'}]",508e4b2ae4b01ceeb4ae3d37,Arabian Tea House Cafe (كافية بيت الشاي العربي),Al Fahidi St,25.263399,55.299695,"[{'label': 'display', 'lat': 25.26339894786175, 'lng': 55.29969492240867}]",1551,AE,Al Souk Al Kabir,دبي,دبي,الإمارات العربية المتحدة,"[Al Fahidi St, دبي, الإمارات العربية المتحدة]","[{'id': '4bf58dd8d48988d1dc931735', 'name': 'Tea Room', 'pluralName': 'Tea Rooms', 'shortName': 'Tea Room', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/tearoom_', 'suffix': '.png'}, 'primary': True}]",0,[],NaN,NaN,NaN
1,e-0-4b7d809af964a520c2c22fe3-1,0,"[{'summary': 'This spot is popular', 'type': 'general', 'reasonName': 'globalInteractionReason'}]",4b7d809af964a520c2c22fe3,Special Ostadi Kabab (مطعم الأستاد للكباب الخاص),At Mankhool Rd,25.259961,55.298940,"[{'label': 'display', 'lat': 25.259961, 'lng': 55.29894}]",1914,AE,Al Hamriya,دبي,دبي,الإمارات العربية المتحدة,"[At Mankhool Rd (Al Monirah St), دبي, الإمارات العربية المتحدة]","[{'id': '52e81612bcbc57f1066b79f7', 'name': 'Persian Restaurant', 'pluralName': 'Persian Restaurants', 'shortName': 'Persian', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/middleeastern_', 'suffix': '.png'}, 'primary': True}]",0,[],Al Monirah St,NaN,NaN
2,e-0-5866b18245c3ed1e7db9931d-2,0,"[{'summary': 'This spot is popular', 'type': 'general', 'reasonName': 'globalInteractionReason'}]",5866b18245c3ed1e7db9931d,Nikki Beach Club,Pearl Jumeirah,25.248306,55.256939,"[{'label': 'display', 'lat': 25.248305850634996, 'lng': 55.25693893432617}]",5084,AE,NaN,دبي,دبي,الإمارات العربية المتحدة,"[Pearl Jumeirah, دبي, الإمارات العربية المتحدة]","[{'id': '52e81612bcbc57f1066b7a0d', 'name': 'Beach Bar', 'pluralName': 'Beach Bars', 'shortName': 'Beach Bar', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/nightlife/default_', 'suffix': '.png'}, 'primary': True}]",0,[],NaN,NaN,NaN


#### Create Categories Column

In [27]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [28]:
venues_df['categories'] = venues_df.apply(get_category_type, axis=1)

In [29]:
venues_df.head(3)

,referralId,reasons.count,reasons.items,venue.id,venue.name,venue.location.address,venue.location.lat,venue.location.lng,venue.location.labeledLatLngs,venue.location.distance,venue.location.cc,venue.location.neighborhood,venue.location.city,venue.location.state,venue.location.country,venue.location.formattedAddress,venue.categories,venue.photos.count,venue.photos.groups,venue.location.crossStreet,venue.location.postalCode,venue.venuePage.id,categories
0,e-0-508e4b2ae4b01ceeb4ae3d37-0,0,"[{'summary': 'This spot is popular', 'type': 'general', 'reasonName': 'globalInteractionReason'}]",508e4b2ae4b01ceeb4ae3d37,Arabian Tea House Cafe (كافية بيت الشاي العربي),Al Fahidi St,25.263399,55.299695,"[{'label': 'display', 'lat': 25.26339894786175, 'lng': 55.29969492240867}]",1551,AE,Al Souk Al Kabir,دبي,دبي,الإمارات العربية المتحدة,"[Al Fahidi St, دبي, الإمارات العربية المتحدة]","[{'id': '4bf58dd8d48988d1dc931735', 'name': 'Tea Room', 'pluralName': 'Tea Rooms', 'shortName': 'Tea Room', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/tearoom_', 'suffix': '.png'}, 'primary': True}]",0,[],NaN,NaN,NaN,Tea Room
1,e-0-4b7d809af964a520c2c22fe3-1,0,"[{'summary': 'This spot is popular', 'type': 'general', 'reasonName': 'globalInteractionReason'}]",4b7d809af964a520c2c22fe3,Special Ostadi Kabab (مطعم الأستاد للكباب الخاص),At Mankhool Rd,25.259961,55.298940,"[{'label': 'display', 'lat': 25.259961, 'lng': 55.29894}]",1914,AE,Al Hamriya,دبي,دبي,الإمارات العربية المتحدة,"[At Mankhool Rd (Al Monirah St), دبي, الإمارات العربية المتحدة]","[{'id': '52e81612bcbc57f1066b79f7', 'name': 'Persian Restaurant', 'pluralName': 'Persian Restaurants', 'shortName': 'Persian', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/middleeastern_', 'suffix': '.png'}, 'primary': True}]",0,[],Al Monirah St,NaN,NaN,Persian Restaurant
2,e-0-5866b18245c3ed1e7db9931d-2,0,"[{'summary': 'This spot is popular', 'type': 'general', 'reasonName': 'globalInteractionReason'}]",5866b18245c3ed1e7db9931d,Nikki Beach Club,Pearl Jumeirah,25.248306,55.256939,"[{'label': 'display', 'lat': 25.248305850634996, 'lng': 55.25693893432617}]",5084,AE,NaN,دبي,دبي,الإمارات العربية المتحدة,"[Pearl Jumeirah, دبي, الإمارات العربية المتحدة]","[{'id': '52e81612bcbc57f1066b7a0d', 'name': 'Beach Bar', 'pluralName': 'Beach Bars', 'shortName': 'Beach Bar', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/nightlife/default_', 'suffix': '.png'}, 'primary': True}]",0,[],NaN,NaN,NaN,Beach Bar


### Drop Unneeded Columns

In [30]:
venues_df.columns

Index(['referralId', 'reasons.count', 'reasons.items', 'venue.id',
       'venue.name', 'venue.location.address', 'venue.location.lat',
       'venue.location.lng', 'venue.location.labeledLatLngs',
       'venue.location.distance', 'venue.location.cc',
       'venue.location.neighborhood', 'venue.location.city',
       'venue.location.state', 'venue.location.country',
       'venue.location.formattedAddress', 'venue.categories',
       'venue.photos.count', 'venue.photos.groups',
       'venue.location.crossStreet', 'venue.location.postalCode',
       'venue.venuePage.id', 'categories'],
      dtype='object')

In [31]:
keep_columns = [
       'venue.id',
       'venue.name', 'venue.location.address', 
       'venue.location.lat', 'venue.location.lng', 
       'venue.location.cc',
       'venue.location.neighborhood', 'venue.location.city',
       'venue.location.state',
       'categories'
               ]

In [32]:
venues_df = venues_df[keep_columns]

In [33]:
venues_df.head()

,venue.id,venue.name,venue.location.address,venue.location.lat,venue.location.lng,venue.location.cc,venue.location.neighborhood,venue.location.city,venue.location.state,categories
0,508e4b2ae4b01ceeb4ae3d37,Arabian Tea House Cafe (كافية بيت الشاي العربي),Al Fahidi St,25.263399,55.299695,AE,Al Souk Al Kabir,دبي,دبي,Tea Room
1,4b7d809af964a520c2c22fe3,Special Ostadi Kabab (مطعم الأستاد للكباب الخاص),At Mankhool Rd,25.259961,55.298940,AE,Al Hamriya,دبي,دبي,Persian Restaurant
2,5866b18245c3ed1e7db9931d,Nikki Beach Club,Pearl Jumeirah,25.248306,55.256939,AE,NaN,دبي,دبي,Beach Bar
3,556291a3498e3237b9ddf800,Fitness Time (وقت اللياقة),NaN,25.289077,55.347913,AE,NaN,دبي,دبي,Gym
4,5a057657065ef54fbdec2747,The Coffee Lab - Mina Rashid (مختبر القهوة - ميناء راشد),Port Rashid,25.258735,55.277499,AE,NaN,دبي,دبي,Coffee Shop


#### Remove Arabic name from venue names column

In [34]:
venues_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 10 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   venue.id                     100 non-null    object 
 1   venue.name                   100 non-null    object 
 2   venue.location.address       95 non-null     object 
 3   venue.location.lat           100 non-null    float64
 4   venue.location.lng           100 non-null    float64
 5   venue.location.cc            100 non-null    object 
 6   venue.location.neighborhood  18 non-null     object 
 7   venue.location.city          100 non-null    object 
 8   venue.location.state         100 non-null    object 
 9   categories                   100 non-null    object 
dtypes: float64(2), object(8)
memory usage: 7.9+ KB


In [35]:
venues_df['venue_name'] = venues_df['venue.name'].str.replace(r"\(.*\)","")

In [36]:
venues_df.head()

,venue.id,venue.name,venue.location.address,venue.location.lat,venue.location.lng,venue.location.cc,venue.location.neighborhood,venue.location.city,venue.location.state,categories,venue_name
0,508e4b2ae4b01ceeb4ae3d37,Arabian Tea House Cafe (كافية بيت الشاي العربي),Al Fahidi St,25.263399,55.299695,AE,Al Souk Al Kabir,دبي,دبي,Tea Room,Arabian Tea House Cafe
1,4b7d809af964a520c2c22fe3,Special Ostadi Kabab (مطعم الأستاد للكباب الخاص),At Mankhool Rd,25.259961,55.298940,AE,Al Hamriya,دبي,دبي,Persian Restaurant,Special Ostadi Kabab
2,5866b18245c3ed1e7db9931d,Nikki Beach Club,Pearl Jumeirah,25.248306,55.256939,AE,NaN,دبي,دبي,Beach Bar,Nikki Beach Club
3,556291a3498e3237b9ddf800,Fitness Time (وقت اللياقة),NaN,25.289077,55.347913,AE,NaN,دبي,دبي,Gym,Fitness Time
4,5a057657065ef54fbdec2747,The Coffee Lab - Mina Rashid (مختبر القهوة - ميناء راشد),Port Rashid,25.258735,55.277499,AE,NaN,دبي,دبي,Coffee Shop,The Coffee Lab - Mina Rashid


#### Change City and Sate name from Arabic to English

In [37]:
venues_df['venue.location.city'].unique()

array(['دبي', 'Dubai 1', 'وسط مدينة دبي', 'الشارقة'], dtype=object)

In [38]:
pattern = '|'.join(['دبي', 'Dubai 1', 'وسط مدينة دبي', 'Jumeirah 1'])

In [39]:
venues_df['city'] = venues_df['venue.location.city'].str.replace(pattern, 'Dubai')

In [40]:
venues_df['state'] = venues_df['venue.location.state'].str.replace(pattern, 'Dubai')

In [41]:
venues_df.head()

,venue.id,venue.name,venue.location.address,venue.location.lat,venue.location.lng,venue.location.cc,venue.location.neighborhood,venue.location.city,venue.location.state,categories,venue_name,city,state
0,508e4b2ae4b01ceeb4ae3d37,Arabian Tea House Cafe (كافية بيت الشاي العربي),Al Fahidi St,25.263399,55.299695,AE,Al Souk Al Kabir,دبي,دبي,Tea Room,Arabian Tea House Cafe,Dubai,Dubai
1,4b7d809af964a520c2c22fe3,Special Ostadi Kabab (مطعم الأستاد للكباب الخاص),At Mankhool Rd,25.259961,55.298940,AE,Al Hamriya,دبي,دبي,Persian Restaurant,Special Ostadi Kabab,Dubai,Dubai
2,5866b18245c3ed1e7db9931d,Nikki Beach Club,Pearl Jumeirah,25.248306,55.256939,AE,NaN,دبي,دبي,Beach Bar,Nikki Beach Club,Dubai,Dubai
3,556291a3498e3237b9ddf800,Fitness Time (وقت اللياقة),NaN,25.289077,55.347913,AE,NaN,دبي,دبي,Gym,Fitness Time,Dubai,Dubai
4,5a057657065ef54fbdec2747,The Coffee Lab - Mina Rashid (مختبر القهوة - ميناء راشد),Port Rashid,25.258735,55.277499,AE,NaN,دبي,دبي,Coffee Shop,The Coffee Lab - Mina Rashid,Dubai,Dubai


#### Rename Columns

In [42]:
venues_df.columns

Index(['venue.id', 'venue.name', 'venue.location.address',
       'venue.location.lat', 'venue.location.lng', 'venue.location.cc',
       'venue.location.neighborhood', 'venue.location.city',
       'venue.location.state', 'categories', 'venue_name', 'city', 'state'],
      dtype='object')

In [43]:
venues_df = venues_df.rename(columns = {
    'venue.id': 'venue_id',
    'venue.location.address': 'address',
    'venue.location.cc': 'country',
    'venue.location.neighborhood': 'neighborhood',
     })

In [44]:
venues_df.head(2)

,venue_id,venue.name,address,venue.location.lat,venue.location.lng,country,neighborhood,venue.location.city,venue.location.state,categories,venue_name,city,state
0,508e4b2ae4b01ceeb4ae3d37,Arabian Tea House Cafe (كافية بيت الشاي العربي),Al Fahidi St,25.263399,55.299695,AE,Al Souk Al Kabir,دبي,دبي,Tea Room,Arabian Tea House Cafe,Dubai,Dubai
1,4b7d809af964a520c2c22fe3,Special Ostadi Kabab (مطعم الأستاد للكباب الخاص),At Mankhool Rd,25.259961,55.298940,AE,Al Hamriya,دبي,دبي,Persian Restaurant,Special Ostadi Kabab,Dubai,Dubai


#### Finalize Columns

In [45]:
venues_df.columns

Index(['venue_id', 'venue.name', 'address', 'venue.location.lat',
       'venue.location.lng', 'country', 'neighborhood', 'venue.location.city',
       'venue.location.state', 'categories', 'venue_name', 'city', 'state'],
      dtype='object')

In [46]:
keep_columns = ['venue_id', 'venue_name', 'categories', 'city', 'state','neighborhood', 'address', 'venue.location.lat',
       'venue.location.lng', 'country']

In [47]:
venues_df = venues_df[keep_columns]

In [48]:
venues_df.head(2)

,venue_id,venue_name,categories,city,state,neighborhood,address,venue.location.lat,venue.location.lng,country
0,508e4b2ae4b01ceeb4ae3d37,Arabian Tea House Cafe,Tea Room,Dubai,Dubai,Al Souk Al Kabir,Al Fahidi St,25.263399,55.299695,AE
1,4b7d809af964a520c2c22fe3,Special Ostadi Kabab,Persian Restaurant,Dubai,Dubai,Al Hamriya,At Mankhool Rd,25.259961,55.298940,AE


In [49]:
venues_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   venue_id            100 non-null    object 
 1   venue_name          100 non-null    object 
 2   categories          100 non-null    object 
 3   city                100 non-null    object 
 4   state               100 non-null    object 
 5   neighborhood        18 non-null     object 
 6   address             95 non-null     object 
 7   venue.location.lat  100 non-null    float64
 8   venue.location.lng  100 non-null    float64
 9   country             100 non-null    object 
dtypes: float64(2), object(8)
memory usage: 7.9+ KB


### Final Cleanup

##### Remove venue not in Dubai

In [50]:
to_remove = venues_df.loc[venues_df['state'] == 'الشارقة']
to_remove

,venue_id,venue_name,categories,city,state,neighborhood,address,venue.location.lat,venue.location.lng,country
85,4b0587f1f964a52096a822e3,Al Qasba,Plaza,الشارقة,الشارقة,NaN,Taawun Rd,25.322411,55.376334,AE


In [51]:
venues_df = venues_df.drop(to_remove.index)

In [52]:
venues_df.loc[venues_df['state'] == 'الشارقة']

,venue_id,venue_name,categories,city,state,neighborhood,address,venue.location.lat,venue.location.lng,country


### Save Dataset as CSV

In [53]:
path = 'foursquare_dubai_venues.csv'
venues_df.to_csv(path, index=False)

## Merge Venues and Communities Datasets

In [59]:
df_merged = pd.merge(wiki_df, venues_df, left_on='Community Name', right_on='neighborhood', how='outer')

In [60]:
df_merged

,Community Number,Community Name,Area(km2),venue_id,venue_name,categories,city,state,neighborhood,address,venue.location.lat,venue.location.lng,country
0,126.0,Abu Hail,1.27,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,711.0,Al Awir First,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,721.0,Al Awir Second,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,283.0,Aleyas,162.40,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,333.0,Al Bada'a,0.82,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
237,NaN,NaN,NaN,5a12d3fee179100ce19c1778,BVLGARI Resort Dubai,Hotel,Dubai,Dubai,Jumeirah 2,Jumeirah Bay Island,25.210697,55.236428,AE
238,NaN,NaN,NaN,519080d9498ef8b46e92e052,Four Seasons Resort Dubai at Jumeirah Beach,Resort,Dubai,Dubai,Jumeirah 2,Jumeirah Beach Rd,25.202167,55.240171,AE
239,NaN,NaN,NaN,5482d623498e688230ec1f3a,Coya,Peruvian Restaurant,Dubai,Dubai,Jumeirah 2,Jumeirah Beach Rd,25.202581,55.240430,AE
240,NaN,NaN,NaN,4ef993dd9adf7bbf8a73a45d,Platinum Movie Suites,Multiplex,Dubai,Dubai,وسط مدينة دبي,The Dubai Mall,25.196083,55.281314,AE


# Observation 
As there is no common key between the two datasets, Merging is not feasible. The communities dataset will not be used for modeling.